# Data cleaning for Demographics dataset

In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
filename = "test.xlsx"
data = pd.read_excel(filename)

In [3]:
data_new = data[['GeogName','GeoID','Borough','Hsp1E','NHspE','WtNHE','BlNHE','AIANNHE','AsnNHE','NHPINHE','OthNHE','Rc2plNHE']]

Renaming the coloumns 

In [4]:
data_new.rename(columns={'Hsp1E':'Hispanics',
                          'NHspE':'Not Hispanic',
                          'WtNHE':'White alone',
                         'BlNHE':'Black',
                         'AIANNHE':'American Indian',
                         'AsnNHE':'Asian',
                         'OthNHE':'Some other race',
                         'Rc2plNHE':'Two or more races'}, 
                 inplace=True)

C:\Users\kia08\Anaconda3\lib\site-packages\pandas\core\frame.py:3781: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


Dropping the 'Non Hispanic' category

In [5]:
data_new1 = data_new.drop(['Not Hispanic'],axis=1)



Sum of all races

In [6]:
list_sum = data_new1.sum(axis = 1, skipna = True)

Creating an 'Unknown' category to align with the shooting dataset

In [7]:
col_list= list(data_new1)
col_list.remove('Hispanics')
col_list.remove('White alone')
col_list.remove('Black')
col_list.remove('American Indian')
col_list.remove('Asian')

In [8]:
data_new1['Unknown'] = data_new1[col_list].sum(axis=1)
percentage_series= data_new1['GeogName']
percentage_demo = percentage_series.to_frame()

In [9]:
data_new2 = data_new1.drop(['NHPINHE','Some other race','Two or more races'],axis=1)
#data_new2

Creating a table with percentage dataset

In [10]:
percentage_demo['GeoID']= data_new1['GeoID']
percentage_demo['Borough'] = data_new1['Borough']

percentage_demo['White %']= data_new1['White alone']/list_sum*100
percentage_demo['Hispanics %'] = data_new1['Hispanics']/list_sum*100
percentage_demo['Black %'] = data_new1['Black']/list_sum*100
percentage_demo['American Indian %'] = data_new1['American Indian']/list_sum*100
percentage_demo['Asian %'] = data_new1['Asian']/list_sum*100
percentage_demo['Unknown %'] = data_new1['Unknown']/list_sum*100
#percentage_demo.rename(columns = {0:'White %'},inplace=True)

In [11]:
percentage_demo.to_csv('percentage_demos.csv',index=False)

In [12]:
percentage_demo.head()
df = data_new2
#df

In [13]:
pr_to_NTA = pd.read_csv('pr_to_NTA.csv')
pr_to_NTA.head()

,Precinct,NTAName
0,1,Battery Park City-Lower Manhattan
1,1,Chinatown
2,1,SoHo-TriBeCa-Civic Center-Little Italy
3,1,West Village
4,1,park-cemetery-etc-Manhattan


Initialization of variables

In [14]:
Temp = df.loc[1]
Temp = Temp.to_frame()
Temp = Temp.transpose()
Temp = Temp.iloc[0:0]
Precinct_data = Temp.drop(['GeogName', 'GeoID','Borough'], axis=1)
Precinct_data['Precinct'] = 1
precinct = pr_to_NTA.loc[0,'Precinct']
Temp3 = Precinct_data

From NTA to Police district

In [15]:
for i in range(1,len(pr_to_NTA)):
    NTA = pr_to_NTA.loc[i,'NTAName']
    idx = df.loc[df.isin([NTA]).any(axis=1)].index.tolist()
    if pr_to_NTA.loc[i,'Precinct'] == precinct:
        Temp = Temp.append(df.loc[idx])
    else: 
        Temp = Temp.reset_index(drop = True)
        Temp2 = Temp.drop(['GeogName', 'GeoID','Borough'], axis=1)
        Temp2 = Temp2.sum(axis = 0, skipna = True)
        Temp2 = Temp2.to_frame()
        Temp2 = Temp2.transpose()
        Temp3 = Temp3.append(Temp2)
        Temp3['Precinct'] = precinct
        Precinct_data = Precinct_data.append(Temp3)
        Temp = Temp.iloc[0:0]
        Temp3 = Temp3.iloc[0:0]
        Temp = Temp.append(df.loc[idx])
        precinct = pr_to_NTA.loc[i,'Precinct']
        #df.sum(axis = 0, skipna = True)     

C:\Users\kia08\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


Including a sum column

In [16]:
Precinct_data = Precinct_data.reset_index(drop = True)
Precinct_data
Temp_Dataframe=Precinct_data.drop(['Precinct'],axis=1) 
Precinct_data['Total'] = Temp_Dataframe.sum(axis = 1, skipna = True)


Calculating percentage

In [17]:
Precinct_data_percent = Precinct_data.iloc[0:0]
#Precinct_data_percent = Precinct_data_percent.drop(['Total'],axis=1)  
for i in range(len(Precinct_data)):
    temp = Precinct_data.loc[i,:]/Precinct_data.loc[i,'Total']*100
    Precinct_data_percent = Precinct_data_percent.append(temp)

Renaming coloumns

In [18]:
Precinct_data_percent['Precinct'] = Precinct_data.loc[:,'Precinct']
Precinct_data_percent
Precinct_data_percent = Precinct_data_percent.drop(['Total'],axis=1)
Precinct_data_percent.rename(columns={'Hispanics':'Hispanics_%',
                          'White alone':'White_alone_%',
                         'Black':'Black_%',
                         'American Indian':'American_Indian_%',
                         'Asian':'Asian_%',
                         'Unknown':'Unknown_%'}, 
                 inplace=True)


In [19]:
Precinct_data_percent.to_csv('precinct_data_percent.csv',index=False)